In [5]:
# Loading DATA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load your EEG data
df = pd.read_csv('features_raw.csv')
print(f"Data shape: {df.shape}")
print(f"Channels: {df.columns.tolist()}")

Data shape: (8064, 33)
Channels: ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2', 'Unnamed: 32']


In [9]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class EEGNet(nn.Module):
    def __init__(self, n_channels=32, n_timepoints=256, n_classes=2):
        super(EEGNet, self).__init__()
        
        # Conv layer 1: temporal filters
        self.conv1 = nn.Conv2d(
            in_channels=1, 
            out_channels=16, 
            kernel_size=(1, 15),  # filter across time
            padding=(0, 7)
        )
        self.bn1 = nn.BatchNorm2d(16)
        
        # Conv layer 2: spatial filters across channels
        self.conv2 = nn.Conv2d(
            in_channels=16, 
            out_channels=32, 
            kernel_size=(n_channels, 1)  # filter across channels
        )
        self.bn2 = nn.BatchNorm2d(32)
        
        # Pooling
        self.pool = nn.AvgPool2d(kernel_size=(1, 4))
        
        # Fully connected layer
        self.fc1 = nn.Linear(32 * (n_timepoints // 4), n_classes)

    def forward(self, x):
        # x shape: (batch, 1, channels, timepoints)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)  # reduce time dimension
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc1(x)
        return x

# Example usage
model = EEGNet(n_channels=32, n_timepoints=256, n_classes=2)
print(model)

# Dummy input: batch of 8 samples, each with 32 channels × 256 timepoints
dummy_input = torch.randn(8, 1, 32, 256)
output = model(dummy_input)
print("Output shape:", output.shape)  # (batch, n_classes)


EEGNet(
  (conv1): Conv2d(1, 16, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(32, 1), stride=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
  (fc1): Linear(in_features=2048, out_features=2, bias=True)
)
Output shape: torch.Size([8, 2])
